## Setup

In [1]:
#Import Functions necessary for Dashboard
import dash
import datetime
from dash import html, dcc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import base64

## Data Preparation

In [2]:
#Import Data for Dashboard from CSV File, Add population-based columns

#Saved file from https://covid.ourworldindata.org/data/owid-covid-data.csv saved in Notebook folder
data = pd.read_csv("owid-covid-data.csv",sep=',')
   
#Add column of cases by population
data['cases_per_pop']=data['total_cases']/data['population']
#Add column of vaccinations by population (Fully Vaccinated)
data['vaxd_per_pop']=data['people_fully_vaccinated']/data['population']
#Add column of new cases (smoothed) by population
data['new_per_pop']=data['new_cases_smoothed']/data['population']
#Add column of total deaths per capita
data['death_per_pop']=data['total_deaths']/data['population']
#make separate country list for later use
countrylist=data['location'].unique()

In [3]:
#Create Basic blank figure to show at beginning
fig=go.Figure()

## Dashboard Setup

In [4]:
#Basic Dash App Setup
app = dash.Dash()
app.title='COVID Dashboard'
app._favicon = ("covid_icon.png")

#Title Image Setup - opened in Layout
image_filename = 'covid-title-image.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

In [5]:
#Main Layout
app.layout = html.Div([
    #Title Block
    html.Div(children=[
        html.Div([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))],
                 style=dict(width='30%')), #Title Image
        html.Div([html.H1('COVID-19 Pandemic Dashboard',
                          style={"text-align": "center","offset":1})],style=dict(width='50%')), #Title
        html.Div([html.H4('Created By Maxwell Bald',
                          style={"text-align":"right","offset":1})],style=dict(width='20%')) #Name
    ],style=dict(display='flex',justifyContent='center')),
    
    #Label Block
        html.Div(children=[
            html.Div([
                html.Label(['Select Countries:'], style={'font-weight': 'bold', "text-align": "center","offset":1}),
                ],style=dict(width='50%')),
            html.Div([
                html.Label(['Select Plot Data:'], style={'font-weight': 'bold', "text-align": "right","offset":1}),
                ],style=dict(width='50%')),
        ],style=dict(display='flex',justifyContent='center')),
    
    #Dropdown Block - Dropdowns in line
    html.Div(children=[
        #Country Dropdown
        #html.Label('Select Countries',),
        dcc.Dropdown(countrylist, id='country_drop_down', 
                     style=dict(width='80%'),
                     value=['Germany'],
                     multi=True),
        html.Div(id='selected_country'),
    
        #Graph Selection Dropdown
        
        dcc.Dropdown(
            id='statistic',
            style=dict(width='80%'),
            options=[
                {'label':'Cases per Capita', 'value':'cases_per_pop'},
                {'label':'Vaccinations per Capita', 'value':'vaxd_per_pop'},
                {'label':'New Cases Per Capita','value':'new_per_pop'},
                {'label':'Reproduction Value (R)', 'value':'reproduction_rate'},
                {'label':'Deaths per Capita','value':'death_per_pop'} 
            ],
            value = ['cases_per_pop']
        ),
        
    ],style=dict(display='flex')),
   #Figure Block
    html.Div(children=[
        #Figure
        dcc.Graph(id='selected_plot', style={'height': '80vh'}) #Height 80% of view works well in Chrome
    ])
    
])

## Dashboard Functions

In [6]:
#Dashboard Callback
from dash.dependencies import Input, Output

@app.callback(
    Output(component_id='selected_plot',component_property='figure'),
    Input(component_id='country_drop_down',component_property='value'),
    Input(component_id='statistic',component_property='value'),
    prevent_initial_call=True
)
     
def update_graph(country_drop_down,statistic):
    data_comb=pd.DataFrame()
    
    #Edit data to have each country's data in the dataframe for plotting
    for x in country_drop_down: 

        data_x = data[data['location']==x]
        data_x = data_x.reset_index(drop=True)
        
        data_comb=pd.concat([data_comb,data_x])
    
    #Make plot of data depending on the user selection
    if statistic== 'cases_per_pop':
        fig=px.line(data_comb,x="date",y="cases_per_pop", color='location', title= 'COVID-19 Cases by Population', 
            labels={"date":"Date","cases_per_pop":"Cases per Capita","location":"Country Name"})
    elif statistic=='vaxd_per_pop':
        fig=px.line(data_comb,x="date",y="vaxd_per_pop", color='location', title= 'COVID-19 Vaccinations by Population', 
            labels={"date":"Date","vaxd_per_pop":"Full Vaccinations per Capita","location":"Country Name"})
    elif statistic=='new_per_pop':
        fig=px.line(data_comb,x="date",y="new_per_pop", color='location', title= 'New COVID-19 Infections by Population', 
            labels={"date":"Date","new_per_pop":"New Infections per Capita","location":"Country Name"})
    elif statistic=='reproduction_rate':
        fig=px.line(data_comb,x="date",y="reproduction_rate", color='location', title= 'COVID-19 Reproduction Value (R)', 
            labels={"date":"Date","reproduction_rate":"Reproduction Value","location":"Country Name"})
    elif statistic=='death_per_pop':
        fig=px.line(data_comb,x="date",y="death_per_pop", color='location', title= 'COVID-19 Deaths by Population', 
            labels={"date":"Date","death_per_pop":"COVID-19 Deaths per Capita","location":"Country Name"})
    
    #Edit Figure Formatting
    fig.update_layout(xaxis_rangeslider_visible=True) #allow date range to be varied
    fig.update_yaxes(automargin=True)
    fig.update_xaxes(nticks=20,rangeslider_thickness=0.05,rangeslider_bgcolor="green")
    fig.update_layout(title_x=0.5 , title_font_color="red")
    
    return fig
            

## Launch App

In [ ]:
app.run_server(debug=True,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
